# Assignment 2: Decision Trees

In this assignment, you are going to implement a decision tree (or random forest) to forcast the weather.

## Description

- You must implement a `Model` class for training and prediction:
  ```python
  X, y = load_dataset()

  model = Model(num_features, num_classes)

  # training
  model.fit(X, y)

  # prediction
  y_pred = model.predict(X)
  ```
- Please search (Ctrl+F) for `TODO` to see what you need to do. You have to implement the classifier from scratch (do not directly use the classifier in scikit-learn).
- About the dataset
  - Given the **training set**, please **train/validate** on it.  
  (note that your model will get bad testing score if it overfits on the training set)
  - After submitting the assignment, we will train on the same training set and test on the hidden **testing set** for scoring (using [f1-score](https://towardsdatascience.com/a-look-at-precision-recall-and-f1-score-36b5fd0dd3ec#11b8)).

### Typical performance

- **Random Guess**  
  F1-Score: 0.30  
  Accuracy: 0.50
- **Always Predict 1**  
  F1-Score: 0.37  
  Accuracy: 0.22
- **Always Predict 0**  
  F1-Score: 0.00  
  Accuracy: 0.77
- **sklearn.tree.DecisionTreeClassifier**  
  - **Training (5-fold cross-validation mean)**  
    F1-Score: 0.63-0.99  
    Accuracy: 0.85-0.99
  - **Validation (5-fold cross-validation mean)**  
    F1-Score: 0.50-0.60  
    Accuracy: 0.75-0.90


In [5]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

import os
import urllib.request
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score


def load_dataset(url):
  """ Get and load weather dataset. """

  path = url.split('/')[-1] # get the file name from url

  if not os.path.exists(path):
    print('Download:', url)
    urllib.request.urlretrieve(url, path)

  return pd.read_pickle(path) # pickle protocol=4


def get_input_target(df):
  """ Get X and y from weather dataset. """
  
  target_column = 'RainTomorrow' # predict 1 of it rains tomorrow

  X = df.drop(columns=[target_column]).to_numpy()
  y = df[target_column].to_numpy()

  return X, y


def k_fold_cv(model_create_fn, X, y, k=5):
  """ Run k-fold cross-validation. """

  results = []

  idxs = list(range(X.shape[0]))
  np.random.shuffle(idxs)
  X, y = X[idxs], y[idxs]

  for i, (train_idxs, val_idxs) in enumerate(KFold(k).split(idxs)):
    splits = {'train': (X[train_idxs], y[train_idxs]),
              'val':   (X[val_idxs],   y[val_idxs]  )}

    print('Run {}:'.format(i+1))

    model = model_create_fn()
    model.fit(*splits['train']) # training

    for name, (X_split, y_split) in splits.items():
      y_pred = model.predict(X_split)

      result = {'split': name,
                'f1': f1_score(y_pred, y_split),
                'acc': accuracy_score(y_pred, y_split)}
      results.append(result)

      print('{split:>8s}: f1={f1:.4f} acc={acc:.4f}'.format(**result))

  return pd.DataFrame(results)


# @begin

In [6]:

# TODO: you can define or import something here (optional)
import numpy as np

class treenode:
    """Decision tree node"""

    def __init__(self,predicted_class):
      self.predicted_class=predicted_class
      self.feature_index=0
      self.threshold=0
      self.left=None
      self.right=None

class Model:

  def __init__(self, num_features: int, num_classes: int,max_depth=1):
    """
    Initialize the model.

    Args:
        num_features (int) : the input feature size.
        num_classes (int) : number of output classes.
    """

    self.num_features = num_features
    self.num_classes = num_classes

    # TODO: implement your model initialization here (optional)
    self.max_depth=max_depth


  def fit(self, X: np.ndarray, y: np.ndarray):
    """
    Train on input/target pairs.

    Args:
        X (np.ndarray) : training inputs with shape (num_inputs, num_features).
        y (np.ndarray) : training targets with shape (num_inputs,).
    """
    # TODO: implement your training algorithm here
    self.num_classes=len(set(y))
    self.num_features=X.shape[1]
    self.tree=self.grow_tree(X,y)



  def predict(self, X: np.ndarray) -> np.ndarray:
    '''
    Predict y given X.

    Args:
        X (np.ndarray) : inputs, shape: (num_inputs, num_features).
    
    Returns:
        np.ndarray : the predicted integer outputs, shape: (num_inputs,).
    '''

    # TODO: implement your prediction algorithm here

    return [self._predict(inputs) for inputs in X]

  # TODO: define your methods if needed (optional)
  
  def best_split(self,X,y):
    """Find the best split for a node"""

    #at least 2 elements needed
    m=y.size
    if m<=1:
      return None,None

    #count of each class in the current node
    num_parent=[np.sum(y==c) for c in range(self.num_classes)]

    #gini of current node
    best_gini=1.0-sum((n/m)**2 for n in num_parent)
    best_idx,best_thr=None,None

    #loop through all features
    for idx in range(self.num_features):
      #sort data along selected feature
      thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

      num_left=[0]*self.num_classes
      num_right=num_parent.copy()
      for i in range(1,m): #possible split positions
        c=classes[i-1]
        num_left[c]+=1
        num_right[c]-=1
        gini_left=1.0-sum(
          (num_left[x]/i) **2 for x in range(self.num_classes)
        )
        gini_right=1.0-sum(
          (num_right[x]/(m-i)) **2 for x in range(self.num_classes)
        )

        #gini impurity of a split is the weighted average of the gini impurity of the children
        gini=(i*gini_left+(m-i)*gini_right)/m

        if thresholds[i]==thresholds[i-1]:
          continue

        if gini<best_gini:
          best_gini=gini
          best_idx=idx
          best_thr=(thresholds[i]+thresholds[i-1])/2 #midpoint

    return best_idx,best_thr



  def grow_tree(self,X,y,depth=0):
    """Build decision tree recursively"""
    num_samples_per_class=[np.sum(y==i) for i in range(self.num_classes)]
    predicted_class=np.argmax(num_samples_per_class)
    
    node=treenode(predicted_class=predicted_class)
    
    #Split recursively until maximum depth
    if depth < self.max_depth:
      idx,thr=self.best_split(X,y)
      if idx is not None:
        indices_left=X[:, idx] < thr
        X_left,y_left=X[indices_left], y[indices_left]
        X_right,y_right=X[~indices_left], y[~indices_left]
        node.feature_index=idx
        node.threshold=thr
        node.left=self.grow_tree(X_left,y_left,depth+1)
        node.right=self.grow_tree(X_right,y_right,depth+1)
    return node


  def _predict(self,inputs):
    node=self.tree
    while node.left:
      if inputs[node.feature_index]<node.threshold:
        node=node.left
      else:
        node=node.right
    return node.predicted_class


In [7]:
# @end

###########################
# DO NOT CHANGE THIS CELL #
###########################

df = load_dataset('https://lab.djosix.com/weather.pkl')
X_train, y_train = get_input_target(df)

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,RainToday,RainTomorrow
0,7.4,25.100000,0.0,5.469824,7.624853,44.0,4.0,22.0,44.0,25.0,1010.599976,4.43719,4.503167,17.200001,0.0,0
1,12.9,25.700001,0.0,5.469824,7.624853,46.0,19.0,26.0,38.0,30.0,1007.599976,4.43719,2.000000,21.000000,0.0,0
2,9.2,28.000000,0.0,5.469824,7.624853,24.0,11.0,9.0,45.0,16.0,1017.599976,4.43719,4.503167,18.100000,0.0,0
3,17.5,32.299999,1.0,5.469824,7.624853,41.0,7.0,20.0,82.0,33.0,1010.799988,7.00000,8.000000,17.799999,0.0,0
4,14.6,29.700001,0.2,5.469824,7.624853,56.0,19.0,24.0,55.0,23.0,1009.200012,4.43719,4.503167,20.600000,0.0,0


In [8]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

create_model = lambda: Model(X_train.shape[1], 2)
k_fold_cv(create_model, X_train, y_train).groupby('split').mean()

Run 1:
   train: f1=0.5099 acc=0.8185
     val: f1=0.5088 acc=0.8202
Run 2:
   train: f1=0.5156 acc=0.8172
     val: f1=0.5250 acc=0.8160
Run 3:
   train: f1=0.5259 acc=0.8144
     val: f1=0.5225 acc=0.8149
Run 4:
   train: f1=0.5265 acc=0.8145
     val: f1=0.5201 acc=0.8147
Run 5:
   train: f1=0.5260 acc=0.8153
     val: f1=0.5222 acc=0.8116


,f1,acc
split,,
train,0.520793,0.815979
val,0.519732,0.815474


## Submission

1. Make sure your code runs correctly after clicking `"Runtime" > "Restart and run all"`
2. Rename this notebook to `XXXXXXX_2.ipynb`, where `XXXXXXX` is your student ID.
3. Download IPython notebook: `"File" > "Download" > "Download .ipynb"`
4. Download Python source code: `"File" > "Download" > "Download .py"`
5. Create a zip file for `XXXXXXX_2.ipynb` and `XXXXXXX_2.py`  
   named `XXXXXXX_2.zip`, where `XXXXXXX` is your student ID.
6. Upload the zip file to E3.

😊 Good luck!